# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [2]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [4]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 8506ffdf


In [5]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [6]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [7]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [8]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [9]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: c:\Repos\Personal\AIE09Bootcamp\AIE9\07_Deep_Agents\workspace


In [10]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] personalized_sleep_improvement_plan.md (5394 bytes)
[DIR] research
[FILE] sleep_improvement_research_summary.md (13555 bytes)


In [11]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (252 bytes)


In [12]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [13]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: c:\Repos\Personal\AIE09Bootcamp\AIE9\07_Deep_Agents\workspace


In [14]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Perfect! I've created your comprehensive, personalized sleep improvement plan and saved it to `/my_sleep_improvement_plan.md`. 

## 🌙 Your Sleep Plan Summary

Your plan directly addresses your three main challenges:

### **Inconsistent Sleep Schedule (10pm-1am)**
- **Solution:** Establish consistent 10:45 PM bedtime with gradual 15-30 minute adjustments
- **Key Strategy:** Focus on consistent wake time first (7:00-7:30 AM)
- **Timeline:** 2-4 weeks to establish new rhythm

### **Phone Use in Bed** 
- **Solution:** Complete phone-free bedroom policy
- **Implementation:** Set up charging station outside bedroom, use traditional alarm clock
- **Alternatives:** Reading, journaling, gentle stretching, meditation

### **Morning Fatigue**
- **Solution:** Strategic morning light exposure + optimized wake-up protocol
- **Key Actions:** Immediate hydration, 15-30 min bright light, delay caffeine 90-120 minutes
- **Expected Results:** Improved morning energy within 5-6 weeks

## 🎯

In [15]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep improvement strategies (completed)
✅ [todo_5] Create personalized sleep improvement plan (completed)
✅ [todo_7] Save plan to file (completed)
✅ [todo_9] Provide summary and next steps (completed)
✅ [todo_6] Research sleep hygiene best practices (completed)
✅ [todo_8] Research circadian rhythm regulation techniques (completed)
✅ [todo_10] Research blue light and phone use impact on sleep (completed)
✅ [todo_12] Research bedtime routine strategies (completed)
✅ [todo_14] Research morning energy improvement methods (completed)
✅ [todo_16] Structure and compile comprehensive research summary (completed)


Workspace contents:
  [FILE] evidence_based_sleep_improvement_strategies.md (21688 bytes)
  [FILE] my_sleep_improvement_plan.md (7928 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5394 bytes)
  [DIR] research/
  [FILE] sleep_improvement_research_summary.md (13555 by

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
TODO planning with lists of TODO task gives us some sort of presistance and checkpoints what needs to be done, but it can add more latenancy and cost due to extra calls, simple task are then slowed down. 

If todo items is to granulated it can create unecessary noise, if the tasks are to ambigous without checkpoints there is no way to track the progress. 

If the Todos are created but never done, the plan becomes obsolete, after every turn of doing todos there should be check uncompleted todos before if it's done which in the end again goes in a round trip of more cost and latenancy.

If plans changes todos need rewriting due to plan change, which again causes additional costs/latenancy etc.

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
Large documents should have some structure, where agent can then read only section which is needed from the document and not read the whole document. If we're embedding the documents to our vector store, we can then approximately make our document strcture to fit into chunks, add metadata for each chunk like section, category, summary etc.

Must remember conditions I would store in long term memory which would be available for agent to get in .json format for easier availability.

User metrics over time I would store into a db or a file system due to the fact it would bloat longterm memory, which in the end would cause slower retrieval.

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [16]:
### YOUR CODE HERE ###


@tool
def read_file(path: str) -> str:
    """
    Read a text file by a string path
    Only supports .txt files.
    """
    p = Path(path)

    if not p.exists():
        return f"File not found: {path}"

    return p.read_text(encoding="utf-8", errors="replace")


custom_tools.append(read_file)


# Step 3: Create the agent with a research-focused system prompt

research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,
    system_prompt="""
You are a Research Wellness Agent. Your job is to research the user topic using the provided guide
and produce a structured markdown report.

Rules:
- Always start by creating todos for the research workflow.
- You MUST read the primary source using: read_file("data/HealthWellnessGuide.txt")
- Update todo status as you complete each step.
- Save the final report to a markdown file in the workspace (use write_file).

Todo workflow (minimum):
1) Read the guide
2) Extract evidence-based strategies relevant to the topic
3) Write a structured report with at least 5 strategies

Report format (markdown):
# Title
## Summary (5-8 bullets)
## Evidence-based strategies (>=5)
For each strategy include:
- What it is
- Why it works (tie to guide content)
- How to apply (steps)
- Cautions / when not to use
## 7-day starter plan
## References (sections or quotes from the guide)
"""
)


# Step 4: Test with the stress management research task

TODO_STORE.clear()

result = research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."
    }]
})

print(result["messages"][-1].content)


# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")


Perfect! I've successfully created a comprehensive evidence-based stress management guide with 7 detailed strategies. Here's what I've delivered:

## **Comprehensive Stress Management Guide Complete!**

I've created a detailed markdown report saved as `/comprehensive_stress_management_guide.md` that includes:

### **✅ 7 Evidence-Based Strategies (exceeding your minimum of 5):**

1. **Progressive Muscle Relaxation (PMR)** - 14-minute faster stress relief with proven research backing
2. **4-7-8 Breathing Technique** - 60-second activation of parasympathetic nervous system
3. **Mindfulness Meditation** - 42% improvement in stress resilience scores
4. **Morning Exercise Routine** - 12-hour energy boost and 23% better stress management
5. **Gratitude Journaling** - 23% cortisol reduction and 34% mood improvement
6. **Cognitive Restructuring** - 50-70% anxiety symptom reduction through thought reframing
7. **Guided Imagery & Visualization** - 5-10 mmHg blood pressure reduction through mental

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [17]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [18]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [19]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what I've delivered:

## ✅ **Complete Morning Routine Guide Created**

The guide has been saved as `/morning_routine_guide.md` and includes everything you requested:

### 🔬 **Science-Based Research**
- **Circadian rhythm optimization** and how morning activities affect energy
- **Exercise physiology** and the metabolic benefits of morning movement
- **Nutritional timing science** for sustained energy throughout the day
- **Sleep science connections** to morning routine effectiveness
- **Hydration and light exposure research** for alertness and energy

### 📋 **Practical Implementation**
- **5-step morning routine framework** with specific timing (Wake → Hydrate → Move → Fuel → Focus)
- **Detailed exercise recommendations** with 3 different time/intensity options
- **Nutrition guidelines** with specific macro targets and meal examples
- **Mindset practices** includi

In [20]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research morning routine science (completed)
✅ [todo_3] Create comprehensive guide structure (completed)
✅ [todo_5] Save final guide as markdown file (completed)

Generated files in workspace:
  [FILE] 7_day_stress_management_plan.txt (16010 bytes)
  [FILE] comprehensive_stress_management_guide.md (20496 bytes)
  [FILE] comprehensive_stress_management_guide.txt (26177 bytes)
  [FILE] evidence_based_sleep_improvement_strategies.md (21688 bytes)
  [FILE] morning_routine_guide.md (32683 bytes)
  [FILE] my_sleep_improvement_plan.md (7928 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5394 bytes)
  [DIR] research/
  [FILE] sleep_improvement_research_summary.md (13555 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [21]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [22]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [23]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [24]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Great to see you again, Alex! Based on your profile, I can see your primary goal is to improve energy levels, with better sleep as a secondary goal. I also note you prefer morning workouts and have mild anxiety. Let me create a personalized exercise routine that addresses these specific needs.

## Recommended Exercise Routine for Alex

Given your goals of improving energy levels and better sleep, plus your preference for morning workouts and consideration of your mild anxiety, here's what I recommend:

### **Morning Energy-Boosting Routine (30-40 minutes, 4-5 times per week)**

**1. Dynamic Warm-up (5 minutes)**
- Arm circles and leg swings
- Light marching in place
- Gentle spinal twists
- Deep breathing exercises (great for anxiety management)

**2. Cardiovascular Component (15-20 minutes)**
- **Option A:** Brisk walking or light jogging outdoors (natural light helps with energy and sleep cycles)
- **Option B:** Indoor cycling or elliptical at moderate intensity
- **O

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [25]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [26]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [27]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [28]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [29]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Now let me provide you with specific, actionable recommendations based on your assessment:

## Your Wellness Assessment Results

### **Current Wellness State**
You're a health-conscious individual with clear goals facing the common challenges of modern office work. Your vegetarian diet is already a significant health advantage, and having no major health conditions gives you a solid foundation to build upon.

### **Key Findings**
Your three main concerns (sedentary work, sleep issues, and weight management) are interconnected - addressing movement and sleep simultaneously will create positive effects across all areas.

## Immediate Action Items (Start Today)

1. **Set Movement Reminders**
   - Set hourly alarms to stand and move for 2-3 minutes
   - Take phone calls while standing or walking when possible
   - Use stairs instead of elevators

2. **Sleep Environment Setup**
   - Create a "wind-down" routine 1 hour before bed (no screens, dim lighting)
   - Set a consiste

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [30]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli not installed. Install with:
  uv pip install deepagents-cli
  # or
  pip install deepagents-cli


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [31]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [32]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [33]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! I've successfully created your comprehensive 2-week wellness program. Here's what I've delivered for you:

## 🎯 Your Complete Wellness Program

### **Four Reference Files Created:**

1. **`/Alex_Exercise_Plan_2_Week.md`** - Your complete exercise program
   - 3x weekly 30-minute morning routines
   - Progressive structure building from Week 1 to Week 2
   - Specific exercises with form cues and modifications
   - Tips for morning motivation and anxiety management through movement

2. **`/Alex_Vegetarian_Nutrition_Plan_2_Week.md`** - Your vegetarian meal plan
   - Complete 2-week meal plans with recipes
   - Shopping lists and meal prep strategies
   - Energy-boosting and sleep-supporting foods
   - Hydration guidelines and timing recommendations

3. **`/Alex_Stress_Sleep_Management_Plan_2_Week.md`** - Your stress & sleep protocol  
   - Daily stress management techniques
   - Complete sleep optimization system
   - Work boundary strategies
   - Evening

In [34]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create Exercise Plan (completed)
✅ [todo_3] Design Vegetarian Meal Plan (completed)
✅ [todo_5] Develop Stress & Sleep Protocol (completed)
✅ [todo_7] Create Master Wellness Schedule (completed)
✅ [todo_9] Save Individual Plans (completed)
✅ [todo_11] Create Program Overview (completed)

GENERATED FILES
  [FILE] 7_day_stress_management_plan.txt (16010 bytes)
  [FILE] Alex_Complete_2_Week_Wellness_Program.md (10199 bytes)
  [FILE] Alex_Exercise_Plan_2_Week.md (7128 bytes)
  [FILE] alex_stress_management_sleep_plan.txt (4806 bytes)
  [FILE] Alex_Stress_Sleep_Management_Plan_2_Week.md (16838 bytes)
  [FILE] Alex_Vegetarian_Meal_Plan.txt (6728 bytes)
  [FILE] Alex_Vegetarian_Nutrition_Plan_2_Week.md (12908 bytes)
  [FILE] comprehensive_stress_management_guide.md (20496 bytes)
  [FILE] comprehensive_stress_management_guide.txt (26177 bytes)
  [FILE] evidence_based_sleep_improvement_strategies.md (21688 bytes)
  [DIR] exercise_programs/
  [FILE] morning_routine_gu

In [35]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of Alex_Complete_2_Week_Wellness_Program.md:
# Alex's Complete 2-Week Wellness Program
*Integrated Exercise, Nutrition & Stress Management Plan*

## Program Overview
**Duration**: 2 weeks  
**Start Date**: [Fill in your start date]  
**User Profile**: Alex - Vegetarian, mild anxiety, prefers morning exercise  
**Core Goals**: Build consistent exercise routine, improve diet, manage work stress, optimize sleep  

---

## Daily Schedule Integration

### Morning Routine (6:30-8:00 AM)
**6:30 AM - Wake Up & Hydration**
- Consistent wake time every day
- 16-20 oz water immediately upon waking
- 2-minute mindfulness/intention setting

**6:45 AM - Pre-Exercise Mindfulness**
- 5 minutes of breathing exercises and body awareness
- Set positive intention for workout and day
- Review workout plan for the session

**7:00 AM - Exercise (Monday, Wednesday, Friday)**
- 30-minute workout per exercise plan
- Focus on form, consistency, and enjoyment
- Post-workout: 3 minutes quiet reflection



---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
Subagents should share generic tools like read or write in a file. Every subagents has the same logic for read/write. Distinct tools should be when we are isolating capabilites to special subagents, resarch subagent should have his resarch tool that could invoke higher cost api call. Writer agent shoulndt have resarch tool available due to the fact that he can summarize text from output of resarch subagent etc.

Simpler tasks like summarizig, extracting thing from the docs should be given to smaller models, subagents that do high risk task which have to be done by reasarching/thinking/decision making should be output to better models. If we gave every subagent a cheaper models, we would save cost but the complex tasks would lack quality and reasoning. If we opted for higher models, our costs would skyrocket.

We should specialaze our agents by domain, excerscise agent, nutriotion agent, sleep agent etc. too much granularity makes orchestration too complex which in the end makes implementation nightmare. Few subagents with a cordinator is much better than multi coordinators and multisubagents.

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
We should add guardrails for health advice so that we do not diagose health issues, that we handle selfharm messages with a safe response to deescalate the situation. We should check every step before outuput to check conflict checks such as allergies and conditions.

We shoulud have presistant storages like db and vector db for storing embeddings.

Also like we did before we should isolate potential user leaks so that isloations is done per user using namespaces, add additional authentication and authorization etc.

We should monitor using langsmith temeletry, store logs and metric to better analize our latenancy, costs, error rates etc.

We should use cheaper models by default while using expensive ones for complex tasks, implementation of maxtokens per agents, implementing rate limits etc.

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [39]:
### YOUR CODE HERE ###
from datetime import datetime

memory_store = InMemoryStore()
user_id = "user_Chat"

# Store profile
profile_ns = (user_id, "profile")
memory_store.put(profile_ns, "name", {"value": "Chat"})
memory_store.put(profile_ns, "goals", {
    "primary": "reduce stress",
    "secondary": "improve sleep and energy",
})
memory_store.put(profile_ns, "constraints", {
    "exercise_time_per_session_minutes": 30,
    "exercise_sessions_per_week": 3,
})
memory_store.put(profile_ns, "conditions", {
    "allergies": ["peanuts"],
    "medications": ["cetirizine"],
    "notes": ["sensitive to caffeine after 14:00"],
})
memory_store.put(profile_ns, "preferences", {
    "diet": "vegetarian",
    "communication_style": "concise",
    "work_schedule": "office 9-17",
    "sleep_target_hours": 7.5,
})

prefs_ns = (user_id, "preferences")
memory_store.put(prefs_ns, "diet", {"value": "vegetarian"})
memory_store.put(prefs_ns, "exercise_schedule", {"value": "3x per week, 30 minutes"})
memory_store.put(prefs_ns, "focus", {"value": "stress + sleep"})
memory_store.put(prefs_ns, "style", {"value": "concise"})

# Step 1: Define your subagent configurations
exercise_subagent = {
    "name": "exercise-specialist",
    "description": "Creates safe, progressive exercise routines and adaptations based on feedback.",
    "system_prompt": """
You are an exercise specialist.
- Create 30-day plans in progressive weekly blocks.
- Always include warm-up, main work, cooldown, and recovery guidance.
- Provide beginner-friendly alternatives and safety notes.
Return concise structured markdown sections only (no long explanations).
""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

nutrition_subagent = {
    "name": "nutrition-specialist",
    "description": "Creates simple nutrition guidance and meal templates aligned to preferences.",
    "system_prompt": """
You are a nutrition specialist.
- Respect dietary preferences and constraints.
- Produce practical guidance (templates, shopping list ideas, simple meals).
- Avoid medical claims; keep advice general and safe.
Return concise structured markdown sections only.
""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}


mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Creates stress management and sleep practices and adapts based on feedback.",
    "system_prompt": """
You are a mindfulness specialist.
- Provide practical stress techniques (breathing, journaling, boundaries).
- Provide sleep hygiene and wind-down routine guidance.
- Avoid medical claims and diagnosis.
Return concise markdown sections only.
""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

# Step 2: Create any additional tools you need
@tool
def log_daily_checkin(user_id: str, day: int, mood: int, energy: int, sleep_hours: float, notes: str = "") -> str:
    """Store daily check-in notes to workspace/users/{user_id}/checkins.tsv"""
    ts = datetime.utcnow().isoformat()
    line = f"{ts}\tday={day}\tmood={mood}\tenergy={energy}\tsleep={sleep_hours}\tnotes={notes}\n"
    p = workspace_path / "users" / user_id / "checkins.tsv"
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text((p.read_text() if p.exists() else "") + line, encoding="utf-8")
    return f"Logged check-in for {user_id} day {day}"

@tool
def write_weekly_summary(user_id: str, week: int, content: str) -> str:
    """Write weekly summary report markdown to workspace/users/{user_id}/weekly_summary_week_{week}.md"""
    p = workspace_path / "users" / user_id / f"weekly_summary_week_{week}.md"
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(content, encoding="utf-8")
    return f"Wrote weekly summary for {user_id} week {week}"

# Step 3: Build the main coordinator agent
wellness_challenge_coach = create_deep_agent(
    model = init_chat_model("openai:gpt-5.2"),
    tools=[
        write_todos, update_todo, list_todos,
        get_user_profile, save_user_preference,
        log_daily_checkin, write_weekly_summary,
    ],
    backend=filesystem_backend,
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a 30-Day Wellness Challenge Coach.

You MUST use all 4 Deep Agent elements:
1) Planning: create todos for a 30-day wellness challenge and update them as you progress.
2) Context Management: store daily check-in notes using log_daily_checkin().
3) Subagents: delegate exercise to exercise-specialist, nutrition to nutrition-specialist, stress/sleep to mindfulness-specialist.
4) Memory: remember user preferences across interactions using save_user_preference / get_user_preferences.

Workflow:
- If user_id is missing, ask for it.
- At the start: load preferences via get_user_preferences(user_id).
- Create a 30-day plan in 4 weekly blocks + simple daily checklist.
- Save the 30-day plan to a markdown file in workspace using file tools (write_file if available, otherwise include plan in response).
- For daily check-ins: call log_daily_checkin(user_id, day, mood, energy, sleep_hours, notes).
- Each week (days 7/14/21/28): write a weekly summary report and save it using write_weekly_summary().

Keep outputs practical and concise."""
)


# Step 4: Test with a user creating their 30-day challenge

TODO_STORE.clear()

result = wellness_challenge_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """My im Chat.
Preferences: vegetarian, exercise 3x/week for 30 mins.
Goal: better sleep, lower stress, more energy.
Create my 30-day wellness challenge plan and remember my preferences."""
    }]
})
print(result["messages"][-1].content)

print("\nTodos after plan creation:")
print(list_todos.invoke({}))


# Step 5: Simulate a daily check-in and adaptation

result2 = wellness_challenge_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Daily check-in:
user_id=user_Chat
day=1
mood=6
energy=5
sleep_hours=5.5
notes=Stressful day, skipped workout, phone in bed. Adapt tomorrow."""
    }]
})
print(result2["messages"][-1].content)

print("\nStored preferences:")
print(get_user_profile.invoke({"user_id": "user_Chat"}))


Preferences saved for **user_id: Chat**:
- **Vegetarian**
- **Exercise:** 3x/week for **30 minutes**
- **Goals:** better sleep, lower stress, more energy

Your 30-day plan is created and saved here:
- **/workspace/30_day_wellness_challenge_Chat.md**

## What’s inside your plan
- **4 weekly blocks (Days 1–30)** with clear themes (breathing → mindfulness → light/activity rhythm → sleep hygiene)
- **Simple daily checklist** (light, movement, nutrition anchor, caffeine cutoff, stress reset, wind-down, consistent sleep window)
- **Exercise plan**: 3 workouts/week (A/B/C) for 30 minutes with weekly progression
- **Vegetarian nutrition framework**: plate method, staples, meal/snack ideas, sleep-friendly timing
- **Nightly wind-down routine** + a daily check-in template you can send me

## Next step (Day 1)
Reply with your **Day 1 check-in** in this format:
- Day #: 1  
- Mood (1–10):  
- Energy (1–10):  
- Sleep hours:  
- Notes (stress, workout?, caffeine cutoff?, wins/challenges):  

Then I

C:\Users\akatalinic1\AppData\Local\Temp\ipykernel_15380\1611475114.py:84: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat()


Day 1 logged (mood 6/10, energy 5/10, sleep 5.5h). Given the stress + skipped workout + phone in bed, here’s the **Day 2 adaptation** (keep it simple and win momentum):

## Day 2 Plan (adapted)
### 1) Sleep/phone fix (priority)
- **Tonight:** charge phone **outside the bedroom** (or across the room).
- Do **20 minutes screen-free before bed** (we’ll build toward 60).
- If you wake at night: no scrolling—just 6 slow breaths.

### 2) Stress reset (2 minutes, during work)
- Do this once mid-day: **6 slow breaths** (inhale nose 4s, exhale mouth 6s) + stand and stretch.

### 3) Movement (choose A or B)
- **A (if you can do 30 min): Week 1 Workout**
  - Warm-up 5 min
  - Circuit 2x: squats x10, push-ups x5 (knees OK), glute bridges x10, plank 20s
  - Cool down 5 min
- **B (if still wiped):** 20–30 min easy walk + 5 min gentle mobility

### 4) Nutrition (steady energy)
- Aim for **protein + fiber at breakfast and lunch** (e.g., yogurt + fruit, or chickpeas/lentils/tofu + veg).

## Quick quest

In [40]:
result3 = wellness_challenge_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Daily check-in:
user_id=user_Chat
day=2
mood=10
energy=6
sleep_hours=8
notes=Today I did gym session, feeling great, though im a bit tired, skipped phone in bed."""
    }]
})
print(result3["messages"][-1].content)

print("\nStored preferences:")
print(get_user_profile.invoke({"user_id": "user_Chat"}))

C:\Users\akatalinic1\AppData\Local\Temp\ipykernel_15380\1611475114.py:84: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat()


Logged Day 2 check-in for **user_Chat**.

## Day 2 quick feedback
- **Wins:** Gym session + **no phone in bed** (big sleep-quality lever).
- **Mood 10 / Energy 6:** Feeling great but a bit tired is consistent with a solid workout day.
- **Sleep 8h:** Above your 7.5h target—nice.

## Today’s micro-adjustment (to manage tiredness without derailing)
**Recovery focus (10–15 min total):**
1) **Hydrate:** 500–750 ml water in the next hour.
2) **Light movement:** 8–10 min easy walk or gentle cycling.
3) **Downshift:** 3 minutes slow breathing (inhale 4s, exhale 6s).

**Evening (keep the no-phone streak):**
- If you feel wired: do a **5-minute stretch** + dim lights 30–60 min before bed.

## Tiny nutrition nudge (vegetarian + energy)
After gym (or next meal), aim for **protein + carbs**:
- Greek yogurt + fruit, or
- Tofu + rice, or
- Lentil soup + bread.

If you want, tell me what time you trained and what you ate after—I'll fine-tune tomorrow’s energy plan.

Stored preferences:
Profile for us

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)